In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx

In [2]:
#I want to be able to feed this raw twitter data so I am going to start with a unprocessed political subset of data
df = pd.read_csv('/notebooks/Di/data/Pol5.csv')

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (6,33,34) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
#Let's take a look at the df to get an idea of what we are working with
df.head()

,time,id,text,user,uid,hashtag,symbol,mention_username,mention_id,mention_realname,...,rtusername,quote_count,reply_count,retweet_count,favorite_count,lang,rtb,rtcheck,mentioncheck,replycheck
0,2020-05-14 20:20:03+00:00,1261028523772391424,Too many families were already struggling to n...,MurrayCampaign,158470209,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,0,0,en,False,False,False,False
1,2020-05-14 20:20:03+00:00,1261028524795838466,@realDonaldTrump @trish_regan China sent the v...,MariKay77,727153519,NaN,NaN,realDonaldTrump trish_regan,25073877 138182116,Donald J. Trump Trish Regan,...,NaN,0,0,0,0,en,True,False,True,True
2,2020-05-14 20:20:04+00:00,1261028526020538375,@eileeniorio @LotusOak2 @POTUS You are very na...,GlendaMcRose,1370806812,NaN,NaN,eileeniorio LotusOak2 POTUS,2932344645 424664120 822215679726100480,EileenIorio LotusOak President Trump,...,NaN,0,0,0,0,en,True,False,True,False
3,2020-05-14 20:20:04+00:00,1261028529451479048,@lien2marsha @OttoPardey @RepFredKeller @White...,jazbec_mike,965060983646572544,NaN,NaN,lien2marsha OttoPardey RepFredKeller WhiteHous...,3882819740 1260226019287601153 113606076142240...,Marsha Lien Otto Pardey Congressman Fred Kelle...,...,NaN,0,0,0,0,en,True,False,True,False
4,2020-05-14 20:20:06+00:00,1261028537995276294,@ProfFionasm @ChrisMurphyCT last leg delivery ...,thatotherGH,190373454,NaN,NaN,ProfFionasm ChrisMurphyCT,913093166638731264 150078976,Fiona Scott Morton Chris Murphy,...,NaN,0,0,0,0,en,True,False,True,False


In [4]:
df.shape

(320926, 51)

In [7]:
df.columns

Index(['time', 'id', 'text', 'user', 'uid', 'hashtag', 'symbol',
       'mention_username', 'mention_id', 'mention_realname', 'url',
       'expanded_url', 'source', 'in_reply_to_id', 'in_reply_to_uid',
       'in_reply_to_username', 'user_loc', 'user_created_at',
       'user_follower_count', 'user_favorite_count', 'user_friends_count',
       'user_status_count', 'user_listed_count', 'user_description',
       'user_url', 'user_protected', 'user_utc_offset', 'user_timezone',
       'user_geo_enabled', 'user_contributors_enabled', 'user_is_translator',
       'user_verified', 'user_lang', 'geo', 'coordinates', 'place',
       'contributors', 'quote', 'retweet', 'rtid', 'rtuid', 'rtusername',
       'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'lang',
       'rtb', 'rtcheck', 'mentioncheck', 'replycheck'],
      dtype='object')

In [8]:
df_net = df[['id','text','uid','mention_id','in_reply_to_id','in_reply_to_uid','retweet','mentioncheck']]

In [9]:
df_net.head()

,id,text,uid,mention_id,in_reply_to_id,in_reply_to_uid,retweet,mentioncheck
0,1261028523772391424,Too many families were already struggling to n...,158470209,NaN,NaN,NaN,False,False
1,1261028524795838466,@realDonaldTrump @trish_regan China sent the v...,727153519,25073877 138182116,1.260579e+18,2.507388e+07,False,True
2,1261028526020538375,@eileeniorio @LotusOak2 @POTUS You are very na...,1370806812,2932344645 424664120 822215679726100480,1.260990e+18,2.932345e+09,False,True
3,1261028529451479048,@lien2marsha @OttoPardey @RepFredKeller @White...,965060983646572544,3882819740 1260226019287601153 113606076142240...,1.260993e+18,3.882820e+09,False,True
4,1261028537995276294,@ProfFionasm @ChrisMurphyCT last leg delivery ...,190373454,913093166638731264 150078976,1.261015e+18,9.130932e+17,False,True
